In [ ]:
# УДОБНЫЕ КЛАССЫ И ФУНКЦИИ ДЛЯ УПРОЩЕНИЯ РАСЧЕТОВ
from functools import total_ordering


@total_ordering
class Point:
    """Класс точки функции и ее значения для более удобных манипуляций"""

    def __init__(self, point, value):
        self.point = point
        self.value = value

    def __eq__(self, other):
        return self.value == other.value

    def __le__(self, other):
            return self.value <= other.value

# Задание 1

Написать в среде MATLAB функции, реализующие метод перебора, метод поразрядного поиска, метод дихотомии, метод золотого сечения, метод парабол, метод средней точки, метод хорд и метод Ньютона.

In [5]:
# МЕТОД ПЕРЕБОРА

from math import ceil

def iterate_method(func, a, b, eps):
    """Реализация метода перебора. Возвращает точку минимума, значение минимума функции и количество вычислений"""

    segments_number = ceil((b - a)/eps)

    func_results = []
    for point_num in range(segments_number+1):
        func_point = a + point_num*(b-a)/segments_number
        func_results.append(
            (func_point, func(func_point))
        )

    return *min(func_results, key=lambda x: x[1]), segments_number

In [19]:
# МЕТОД ПОРАЗРЯДНОГО ПОИСКА

def bitByBitSearch(func, a, b, eps):
    """Реализация метода поразрядного поиска. Возвращает точку минимума, значение минимума функции и количество вычислений"""

    cur_eps = 0.25 if 0.25 > eps else eps
    left_bound, right_bound = a, b

    direction_to_right = True
    compute_number = 0
    
    min_point = Point(0, float('inf'))
    
    while True:
        segments_number = ceil((right_bound - left_bound)/cur_eps)

        cur_range = range(segments_number+1) if direction_to_right else range(segments_number, -1, -1)

        for point_num in cur_range:
            compute_number += 1
            
            func_arg = left_bound + point_num*(right_bound - left_bound)/segments_number
            func_value = func(func_arg)
            func_point = Point(func_arg, func_value)
            
            if func_point >= min_point:
                break
            else:
                min_point = func_point

        if cur_eps <= eps:
            break
        else:
            direction_to_right = not direction_to_right
            cur_eps /= 4

    return min_point.point, min_point.value, compute_number
                


In [9]:
def dichotomy_minimize(func, a, b, eps):
    
    # Маленькая величина для смещения относительно середины отрезка
    delta = eps / 2

    compute_number = 0
    
    while (b - a)/2 > eps:

        x1, x2 = (a + b - delta) / 2, (a + b + delta) / 2
        compute_number += 1
        
        if func(x1) < func(x2):
            # Минимум находится в левом подотрезке
            b = x2
        else:
            # Минимум находится в правом подотрезке
            a = x1

    minim_value = (a + b) / 2
    final_res = func(minim_value)
    
    # Возвращаем середину финального отрезка как приближение минимума
    return minim_value, final_res, compute_number

In [11]:
from math import sqrt

def golden_section(func, a, b, eps):

    sqrt_5 = sqrt(5)
    tau = (sqrt_5 - 1) / 2
    n = 0
    compute_number = 0
    
    x1, x2 = a + ((3 - sqrt_5) / 2) * (b - a), a + ((sqrt_5 - 1) / 2) * (b - a)
    f1, f2 = func(x1), func(x2)
    compute_number += 2
    eps_n = (b - a) / 2

    while eps_n <= eps:
        if f1 <= f2:
            b = x2
            x2 = x1
            f2 = f1
            x1 = b - tau*(b - a)
            f1 = func(x1)
        else:
            a = x1
            x1 = x2
            f1 = f2
            x2 = b - tau*(b - a)
            f2 = func(x2)
            
        compute_number += 1

    minim_value = (a + b) / 2
    final_res = func(minim_value)

    return minim_value, final_res, compute_number